## STEP 1

In [36]:
import random
random.seed

0

In [37]:
#1 to represent the player we use the class PLayer
# display from https://stackoverflow.com/questions/34012886/print-binary-tree-level-by-level-in-python
class Player:
    def __init__(self,playerId,score):
        self.id=playerId
        self.score=score #average score
        self.nb_games=0  #to calculate average
        self.left=None
        self.right=None
        self.height=1
        
    def __str__(self):
        return f"{self.id} : {self.score} x {self.nb_games}"
    
    #ajouter un score
    def add_score(self,game_score):
        self.score=(self.score*self.nb_games+game_score)/(self.nb_games+1) #update average
        self.nb_games+=1 
        
    def reset_score(self):
        self.score=0
        self.nb_games=0
        
    def reset_childrens(self): #for re inserting in new tree
        self.right=None
        self.left=None
        self.heigth=1
        
    def display(self):
        lines, *_ = self._display_aux()
        for line in lines:
            print(line)

    def _display_aux(self):
        """Returns list of strings, width, height, and horizontal coordinate of the root."""
        # No child.
        if self.right is None and self.left is None:
            line = '%s' % self.score
            width = len(line)
            height = 1
            middle = width // 2
            return [line], width, height, middle

        # Only left child.
        if self.right is None:
            lines, n, p, x = self.left._display_aux()
            s = '%s' % self.score
            u = len(s)
            first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s
            second_line = x * ' ' + '/' + (n - x - 1 + u) * ' '
            shifted_lines = [line + u * ' ' for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, n + u // 2

        # Only right child.
        if self.left is None:
            lines, n, p, x = self.right._display_aux()
            s = '%s' % self.score
            u = len(s)
            first_line = s + x * '_' + (n - x) * ' '
            second_line = (u + x) * ' ' + '\\' + (n - x - 1) * ' '
            shifted_lines = [u * ' ' + line for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, u // 2

        # Two children.
        left, n, p, x = self.left._display_aux()
        right, m, q, y = self.right._display_aux()
        s = '%s' % self.score
        u = len(s)
        first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s + y * '_' + (m - y) * ' '
        second_line = x * ' ' + '/' + (n - x - 1 + u + y) * ' ' + '\\' + (m - y - 1) * ' '
        if p < q:
            left += [n * ' '] * (q - p)
        elif q < p:
            right += [m * ' '] * (p - q)
        zipped_lines = zip(left, right)
        lines = [first_line, second_line] + [a + u * ' ' + b for a, b in zipped_lines]
        return lines, n + m + u, max(p, q) + 2, n + u // 2




In [38]:
#2 we use an AVL tree ranked by score
# At each step of the game the player are mooved to a new tree with a new ranking
class Classement:
    
    #recursive insertion
    def insert(self,root,player):
        #insertion BST
        if not root:
            return(player)
        elif player.score<root.score:
            root.left=self.insert(root.left,player)
        elif player.score>root.score:
            root.right=self.insert(root.right,player)
        elif player.score==root.score: #if equality insert on lowest height side
            if self.getHeight(root.left)>self.getHeight(root.right):
                root.right=self.insert(root.right,player)
            elif self.getHeight(root.left)<self.getHeight(root.right):
                root.left=self.insert(root.left,player)
            elif self.getHeight(root.left)==self.getHeight(root.right): #if equal height choose random side
                side=random.choice(["left","right"])
                if side =="right":
                    root.right=self.insert(root.right,player)
                else :
                    root.left=self.insert(root.left,player)
                
        
        #height update
        root.height=1+max(self.getHeight(root.left),self.getHeight(root.right))
        
        #balance
        balance = self.getBalance(root)
        
        #balancing if necesssary
        if balance > 1 and player.score < root.left.score: 
            return self.rightRotate(root)
        if balance < -1 and player.score > root.right.score: 
            return self.leftRotate(root) 
        if balance > 1 and player.score > root.left.score: 
            root.left = self.leftRotate(root.left) 
            return self.rightRotate(root)
        if balance < -1 and player.score < root.right.score: 
            root.right = self.rightRotate(root.right) 
            return self.leftRotate(root) 
        
        return root
    
    def delete(self, root, player): 
  
        # BST delete 
        if not root: 
            return root 
        elif player.score < root.score: 
            root.left = self.delete(root.left, player) 
        elif player.score > root.score: 
            root.right = self.delete(root.right, player) 
        elif player.score == root.score :
            if player.id != root.id: #si il y a égalité on recherche des 2 cotés 
                root.right = self.delete(root.right, player) 
                root.left = self.delete(root.left, player) 
            else : 
                if root.left is None: 
                    temp = root.right 
                    root = None
                    return temp 

                elif root.right is None: 
                    temp = root.left 
                    root = None
                    return temp 

                temp = self.getMinPlayer(root.right) 
                root.score = temp.score
                root.id=temp.id
                root.nb_games=temp.nb_games
                root.right = self.delete(root.right,temp) 
            
                
        if root is None: 
            return root 
  
        # height update
        root.height = 1 + max(self.getHeight(root.left),self.getHeight(root.right)) 
  
        # balance 
        balance = self.getBalance(root) 
  
        if balance > 1 and self.getBalance(root.left) >= 0: 
            return self.rightRotate(root) 
        if balance < -1 and self.getBalance(root.right) <= 0: 
            return self.leftRotate(root)  
        if balance > 1 and self.getBalance(root.left) < 0: 
            root.left = self.leftRotate(root.left) 
            return self.rightRotate(root) 
        if balance < -1 and self.getBalance(root.right) > 0: 
            root.right = self.rightRotate(root.right) 
            return self.leftRotate(root) 
        
        return root 
    
    def leftRotate(self, z): 
        y = z.right 
        T2 = y.left 
        y.left = z 
        z.right = T2 
        z.height = 1 + max(self.getHeight(z.left),self.getHeight(z.right)) 
        y.height = 1 + max(self.getHeight(y.left),self.getHeight(y.right)) 
        return y 
    
    def rightRotate(self, z): 
        y = z.left 
        T3 = y.right 
        y.right = z 
        z.left = T3  
        z.height = 1 + max(self.getHeight(z.left),self.getHeight(z.right)) 
        y.height = 1 + max(self.getHeight(y.left),self.getHeight(y.right)) 
        return y 
        
    def getHeight(self, root): 
        if not root: 
            return 0
        return root.height 
    
    def getBalance(self, root): 
        if not root: 
            return 0
        return self.getHeight(root.left) - self.getHeight(root.right) 
    
    def getMinPlayer(self, root): 
        if root is None or root.left is None: 
            return root 
        return self.getMinPlayer(root.left) 
    
    def preOrder(self, root): 
        if not root: 
            return
        print(root) 
        self.preOrder(root.left) 
        self.preOrder(root.right) 
        
    def inOrder(self,root): 
        if not root: 
            return
        self.inOrder(root.left) 
        print(root)
        self.inOrder(root.right) 
        
    def returnInOrder(self,root): 
        if not root: 
            return
        self.inOrder(root.left) 
        return(root)
        self.inOrder(root.right) 

In [39]:
tree=Classement()
root=None
print("Insertion Order : ")
for i in range(10):
    p=Player(i,0)
    p.add_score(0)
    print(f"Player : {p}")
    root=tree.insert(root,p)
print("\nTree display :\n")
root.display()

Insertion Order : 
Player : 0 : 0.0 x 1
Player : 1 : 0.0 x 1
Player : 2 : 0.0 x 1
Player : 3 : 0.0 x 1
Player : 4 : 0.0 x 1
Player : 5 : 0.0 x 1
Player : 6 : 0.0 x 1
Player : 7 : 0.0 x 1
Player : 8 : 0.0 x 1
Player : 9 : 0.0 x 1

Tree display :

        _______0.0_______     
       /                 \    
  ____0.0_             _0.0_  
 /        \           /     \ 
0.0_     0.0_       _0.0   0.0
    \        \     /          
   0.0      0.0   0.0         


In [40]:
def playersGen(n,scoreMax=0): #generate n players
    players =[]
    for i in range(n):
        if scoreMax:
            score=random.randint(0,scoreMax)
        else :
            score = 0
        players.append(Player(i,score))
    return players
'''players=playersGen(10)
[print(player) for player in players ]
[(player.add_score(1)) for player in players]
[print(player) for player in players ]'''

'players=playersGen(10)\n[print(player) for player in players ]\n[(player.add_score(1)) for player in players]\n[print(player) for player in players ]'

In [41]:
# 3
# probability based on this data
# https://www.reddit.com/r/AmongUs/comments/glfh8q/my_statistics_after_playing_this_accursed_game/

#A COMPLETER

#simulate score for a game of 10 players
#first generate a game with 10 players then assign the score to each players
def randGame(players_list):   
    # creating the lists:
    #2 impostors
    impostors=[0]*2  
    #the rest are crewmates
    crewmates=[0]*8
    
    #outcome of the game with a 50/50% chance for each teams
    impostorsWin=random.choice([True,False])   
    if impostorsWin:
        # impostors points :
        # +10 points for wining the game
        impostors=[player+10 for player in impostors]
        # +1 point per kill
        nbKillsImpostor1=random.randint(0,8) #game can be won without killing everybody
        nbKillsImpostor2=random.randint(0,8-nbKillsImpostor1)
        impostors[0]+=nbKillsImpostor1
        impostors[1]+=nbKillsImpostor2
        # +3 points per undiscovered murder 
        nbUndicoImpostor1=random.randint(0,nbKillsImpostor1)
        nbUndicoImpostor2=random.randint(0,nbKillsImpostor2)
        impostors[0]+=nbUndicoImpostor1*3
        impostors[1]+=nbUndicoImpostor2*3
        
        # crewmate points :
        # +3 points per unmasked imposter for 1, only 1 imposter can be unmasked because the impostors won
        unmaskedImpostor=random.choice([True,False])
        if unmaskedImpostor:
            crewmates[random.randrange(8)]+=3 
        # +1 point if all solo task made for all crew mates except at least one
        noPoints=random.randrange(8)
        crewmates=[player+(random.choice([0,1])) for player in (crewmates[:noPoints]+crewmates[noPoints+1:])]+[crewmates[noPoints]]
        
    else:
        #impostors points :
        # +1 point per kill
        nbKillsImpostor1=random.randint(0,7)
        nbKillsImpostor2=random.randint(0,7-nbKillsImpostor1)
        impostors[0]+=nbKillsImpostor1
        impostors[1]+=nbKillsImpostor2
        # +3 points per undiscovered murder 
        nbUndicoImpostor1=random.randint(0,nbKillsImpostor1)
        nbUndicoImpostor2=random.randint(0,nbKillsImpostor2)
        impostors[0]+=nbUndicoImpostor1*3
        impostors[1]+=nbUndicoImpostor2*3
    
        # Crewmates points :
        # +5 points for wining the game
        crewmates=[player+5 for player in crewmates]
        
        # 2 ways to win for crewmates : 
        # game can be won by completing all the tasks
        # or by discovering impostors
        win_task=random.choice([True,False]) 
        if win_task : #if the game was won by completing all tasks
            # +3 points per unmasked imposter for up to 1 player
            crewmates[random.randrange(8)]+=random.choice([0,3])      
            # +1 point if all solo task made for every players
            crewmates=[player+1 for player in crewmates]
        else : # if won by eliminating impostors
            # +3 points per unmasked imposter for up to 2 player
            crewmates[random.randrange(8)]+=3
            crewmates[random.randrange(8)]+=3
            # +1 point if all solo task made for all crew mates except at least one
            noPoints=random.randrange(8)
            crewmates=[player+(random.choice([0,1])) for player in (crewmates[:noPoints]+crewmates[noPoints+1:])]+[crewmates[noPoints]]
            
    #assigning new scores :
    scores_list=impostors+crewmates
    scores_list=[12 if score>12 else score for score in scores_list]
    [player.add_score(scores_list.pop(random.randrange(len(scores_list)))) for player in players_list]
    
    return  players_list

In [42]:
players=playersGen(10)
players=randGame(players)
tree=Classement()
root=None
#[print(player) for player in players ]
for p in players:
    root=tree.insert(root,p)
tree.inOrder(root)
print("\n")
while(root!=None):
    minP=tree.getMinPlayer(root)
    print("minP : ",minP)
    root=tree.delete(root,minP)
    tree.inOrder(root)
    print("\n")

9 : 0.0 x 1
4 : 0.0 x 1
2 : 0.0 x 1
3 : 0.0 x 1
6 : 1.0 x 1
0 : 1.0 x 1
1 : 1.0 x 1
5 : 3.0 x 1
7 : 12.0 x 1
8 : 12.0 x 1


minP :  9 : 0.0 x 1
4 : 0.0 x 1
2 : 0.0 x 1
3 : 0.0 x 1
6 : 1.0 x 1
0 : 1.0 x 1
1 : 1.0 x 1
5 : 3.0 x 1
7 : 12.0 x 1
8 : 12.0 x 1


minP :  4 : 0.0 x 1
2 : 0.0 x 1
3 : 0.0 x 1
6 : 1.0 x 1
0 : 1.0 x 1
1 : 1.0 x 1
5 : 3.0 x 1
7 : 12.0 x 1
8 : 12.0 x 1


minP :  2 : 0.0 x 1
3 : 0.0 x 1
6 : 1.0 x 1
0 : 1.0 x 1
1 : 1.0 x 1
5 : 3.0 x 1
7 : 12.0 x 1
8 : 12.0 x 1


minP :  3 : 0.0 x 1
6 : 1.0 x 1
0 : 1.0 x 1
1 : 1.0 x 1
5 : 3.0 x 1
7 : 12.0 x 1
8 : 12.0 x 1


minP :  6 : 1.0 x 1
0 : 1.0 x 1
1 : 1.0 x 1
5 : 3.0 x 1
7 : 12.0 x 1
8 : 12.0 x 1


minP :  0 : 1.0 x 1
1 : 1.0 x 1
5 : 3.0 x 1
7 : 12.0 x 1
8 : 12.0 x 1


minP :  1 : 1.0 x 1
5 : 3.0 x 1
7 : 12.0 x 1
8 : 12.0 x 1


minP :  5 : 3.0 x 1
7 : 12.0 x 1
8 : 12.0 x 1


minP :  7 : 12.0 x 1
8 : 12.0 x 1


minP :  8 : 12.0 x 1




In [43]:
# 5 plays n games :
def nGames(players,n):
    for i in range(n):
        players=randGame(players)
        #print("game : " ,i)
        #[print(player) for player in players ]
    return players
[print(p) for p in nGames(playersGen(10),100) ];

0 : 4.0 x 100
1 : 4.150000000000002 x 100
2 : 4.869999999999999 x 100
3 : 3.6699999999999995 x 100
4 : 4.240000000000002 x 100
5 : 4.2700000000000005 x 100
6 : 5.270000000000002 x 100
7 : 4.800000000000001 x 100
8 : 4.7299999999999995 x 100
9 : 4.059999999999999 x 100


In [44]:
# 4
# at each step of the game the player are removed by batch of 10
# Play their game and are then re inserted in a new tree for the next step
def manche1(n):
    tree = Classement()
    root=None
    for i in range(n): # 10 batchs 
        # création des joueurs
        players=[Player(i*10+j,0) for j in range (10)]
        # 3 premières parties 
        players=nGames(players,3)
        # insertion dans l'arbre 
        for player in players:
            root=tree.insert(root,player)
    #tree.inOrder(root)
    #root.display()
    return root
tree=Classement()
tree.inOrder(manche1(1))

7 : 0.0 x 3
0 : 0.0 x 3
4 : 0.3333333333333333 x 3
1 : 0.3333333333333333 x 3
9 : 0.6666666666666666 x 3
5 : 0.6666666666666666 x 3
2 : 4.0 x 3
3 : 5.333333333333333 x 3
6 : 7.333333333333333 x 3
8 : 9.0 x 3


In [45]:
def manche(root_preced,nb_games=3,final=False):
    tree=Classement()
    
    if not final:     #eliminate 10 last players 
        for i in range(10):
            minPlayer=tree.getMinPlayer(root_preced)
            root_preced=tree.delete(root_preced,minPlayer)

    root_new=None
    tree2=Classement()
    while(root_preced is not None):
        batch=[]
        #creating new games by batch of 10 from ranking
        for i in range(10):
            #print("root",root_preced.right,root_preced.left)
            minPlayer=tree.getMinPlayer(root_preced)
            root_preced=tree.delete(root_preced,minPlayer)
            if final:
                minPlayer.reset_score()
            batch.append(minPlayer)
        batch=nGames(batch,nb_games)
        
        #inserting player in new tree
        for player in batch:
            player.reset_childrens()
            try: 
                root_new=tree.insert(root_new,player)
            except :
                print("error")
                pass
                #root.display()
                #print(player)
            
    return root_new

tree=Classement()
root=manche1(10)
tree.inOrder(root)
print("\n")
root=manche(root)
tree.inOrder(root)
print("\n")
root=manche(root)
tree.inOrder(root)

32 : 0.3333333333333333 x 3
23 : 1.0 x 3
84 : 1.6666666666666667 x 3
40 : 1.6666666666666667 x 3
72 : 2.0 x 3
76 : 2.0 x 3
54 : 2.0 x 3
70 : 2.0 x 3
21 : 2.0 x 3
65 : 2.0 x 3
36 : 2.3333333333333335 x 3
34 : 2.3333333333333335 x 3
43 : 2.3333333333333335 x 3
13 : 2.3333333333333335 x 3
37 : 2.3333333333333335 x 3
35 : 2.3333333333333335 x 3
39 : 2.3333333333333335 x 3
75 : 2.3333333333333335 x 3
18 : 2.6666666666666665 x 3
82 : 2.6666666666666665 x 3
73 : 2.6666666666666665 x 3
88 : 2.6666666666666665 x 3
81 : 3.0 x 3
87 : 3.0 x 3
41 : 3.3333333333333335 x 3
33 : 3.3333333333333335 x 3
64 : 3.6666666666666665 x 3
47 : 3.6666666666666665 x 3
86 : 3.6666666666666665 x 3
17 : 3.6666666666666665 x 3
57 : 4.0 x 3
28 : 4.0 x 3
30 : 4.0 x 3
9 : 4.0 x 3
44 : 4.0 x 3
15 : 4.0 x 3
60 : 4.0 x 3
67 : 4.0 x 3
14 : 4.0 x 3
69 : 4.0 x 3
98 : 4.0 x 3
46 : 4.0 x 3
78 : 4.0 x 3
61 : 4.333333333333333 x 3
97 : 4.333333333333333 x 3
26 : 4.333333333333333 x 3
19 : 4.333333333333333 x 3
56 : 4.333333333333

In [46]:
n=10
tree=Classement()
root=manche1(n)
#tree.inOrder(root)
print("\n")
for i in range(n-1):
    root=manche(root)
#tree.inOrder(root)
root=manche(root,nb_games=5,final=True)
print("\nTOP 10 : ")
tree.inOrder(root)
print("\nPodium : ")
for _ in range(7):
    minPlayer=tree.getMinPlayer(root)
    root=tree.delete(root,minPlayer)
tree.inOrder(root)






TOP 10 : 
74 : 2.4 x 5
60 : 3.2 x 5
4 : 3.6 x 5
71 : 3.8 x 5
54 : 4.4 x 5
72 : 4.6 x 5
45 : 5.2 x 5
90 : 5.8 x 5
6 : 6.0 x 5
98 : 6.0 x 5

Podium : 
90 : 5.8 x 5
6 : 6.0 x 5
98 : 6.0 x 5


# PSEUDO CODE

### Insertion :

### Search :

### Random game :